# Transformer leakage inductance calculation

This example shows how to use PyAEDT to create a Maxwell 2D
magnetostatic analysis to calculate transformer leakage
inductance and reactance.
The analysis based on this document is from page 8 in Professor S. V.
Kulkami's paper, [Basis of Finite Element Method](https://www.ee.iitb.ac.in/~fclab/FEM/FEM1.pdf).

Keywords: **Maxwell 2D**, **transformer**, **motor**.

## Perform imports and define constants

In [1]:
import os
import tempfile
import time

from ansys.aedt.core import Maxwell2d

Define constants,

In [2]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.

## Create temporary directory

Create a temporary directory where downloaded data or
dumped data can be stored.
If you'd like to retrieve the project data for subsequent use,
the temporary folder name is given by ``temp_folder.name``.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Initialize and launch Maxwell 2D

Initialize and launch Maxwell 2D, providing the version, the path to the project, the design
name, and the type.

In [4]:

project_name = os.path.join(temp_folder.name, "Magnetostatic.aedt")
m2d = Maxwell2d(
    version=AEDT_VERSION,
    new_desktop=False,
    design="Transformer_leakage_inductance",
    project=project_name,
    solution_type="MagnetostaticXY",
    non_graphical=NG_MODE,
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


PyAEDT INFO: Log on file C:\Users\ansys\AppData\Local\Temp\pyaedt_ansys_21a25ac8-9fdc-43cd-821f-81389ce905cb.log is enabled.


...

PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.


PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project Magnetostatic has been created.


PyAEDT INFO: Added design 'Transformer_leakage_inductance' of type Maxwell 2D.


PyAEDT INFO: Aedt Objects correctly read


## Initialize dictionaries

Set modeler units and initialize dictionaries
that contain all the definitions for the design variables.

In [5]:
m2d.modeler.model_units = "mm"

dimensions = {
    "core_width": "1097mm",
    "core_height": "2880mm",
    "core_opening_x1": "270mm",
    "core_opening_x2": "557mm",
    "core_opening_y1": "540mm",
    "core_opening_y2": "2340mm",
    "core_opening_width": "core_opening_x2-core_opening_x1",
    "core_opening_height": "core_opening_y2-core_opening_y1",
    "LV_x1": "293mm",
    "LV_x2": "345mm",
    "LV_width": "LV_x2-LV_x1",
    "LV_mean_radius": "LV_x1+LV_width/2",
    "LV_mean_turn_length": "pi*2*LV_mean_radius",
    "LV_y1": "620mm",
    "LV_y2": "2140mm",
    "LV_height": "LV_y2-LV_y1",
    "HV_x1": "394mm",
    "HV_x2": "459mm",
    "HV_width": "HV_x2-HV_x1",
    "HV_mean_radius": "HV_x1+HV_width/2",
    "HV_mean_turn_length": "pi*2*HV_mean_radius",
    "HV_y1": "620mm",
    "HV_y2": "2140mm",
    "HV_height": "HV_y2-HV_y1",
    "HV_LV_gap_radius": "(LV_x2 + HV_x1)/2",
    "HV_LV_gap_length": "pi*2*HV_LV_gap_radius",
}

specifications = {
    "Amp_turns": "135024A",
    "Frequency": "50Hz",
    "HV_turns": "980",
    "HV_current": "Amp_turns/HV_turns",
}

PyAEDT INFO: Modeler2D class has been initialized!


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


## Define variables from dictionaries

Define design variables from the created dictionaries.

In [6]:
m2d.variable_manager.set_variable(name="Dimensions")

for k, v in dimensions.items():
    m2d[k] = v

m2d.variable_manager.set_variable(name="Windings")

for k, v in specifications.items():
    m2d[k] = v

## Create design geometries

Create the transformer core, the HV and LV windings, and the region.

In [7]:
core = m2d.modeler.create_rectangle(
    origin=[0, 0, 0],
    sizes=["core_width", "core_height", 0],
    name="core",
    material="steel_1008",
)

core_hole = m2d.modeler.create_rectangle(
    origin=["core_opening_x1", "core_opening_y1", 0],
    sizes=["core_opening_width", "core_opening_height", 0],
    name="core_hole",
)

m2d.modeler.subtract(blank_list=[core], tool_list=[core_hole], keep_originals=False)

lv = m2d.modeler.create_rectangle(
    origin=["LV_x1", "LV_y1", 0],
    sizes=["LV_width", "LV_height", 0],
    name="LV",
    material="copper",
)

hv = m2d.modeler.create_rectangle(
    origin=["HV_x1", "HV_y1", 0],
    sizes=["HV_width", "HV_height", 0],
    name="HV",
    material="copper",
)

region = m2d.modeler.create_region(pad_percent=[20, 10, 0, 10])

# ## Assign boundary condition
#
# Assign vector potential to zero on all region boundaries. This makes x=0 edge a symmetry boundary.

m2d.assign_vector_potential(assignment=region.edges, boundary="VectorPotential1")

PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Boundary Vector Potential VectorPotential1 has been created.


VectorPotential1

## Create initial mesh settings

Assign a relatively dense mesh to all objects to ensure that the energy is calculated accurately.

In [8]:
m2d.mesh.assign_length_mesh(
    assignment=["core", "Region", "LV", "HV"],
    maximum_length=50,
    maximum_elements=None,
    name="all_objects",
)

PyAEDT INFO: Mesh class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Mesh class has been initialized! Elapsed time: 0m 0sec


all_objects

## Define excitations

Assign the same current in amp-turns but in opposite directions to the HV and LV windings.

In [9]:
m2d.assign_current(assignment=lv, amplitude="Amp_turns", name="LV")
m2d.assign_current(assignment=hv, amplitude="Amp_turns", name="HV", swap_direction=True)

PyAEDT INFO: Boundary Current LV has been created.


PyAEDT INFO: Boundary Current HV has been created.


HV

## Create and analyze setup

Create and analyze the setup. Set the number of minimum passes to 3 to ensure accuracy.

In [10]:
m2d.create_setup(name="Setup1", MinimumPasses=3)
m2d.analyze_setup(use_auto_settings=False, cores=NUM_CORES)

PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


PyAEDT INFO: Solving all design setups. Analysis started...


PyAEDT INFO: Design setup None solved correctly in 0.0h 0.0m 31.0s


PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/Maxwell 2D correctly changed.


True

## Calculate transformer leakage inductance and reactance

Calculate transformer leakage inductance from the magnetic energy with PyAEDT advanced fields calculator.

In [11]:
leakage_inductance = {
    "name": "Leakage_inductance",
    "description": "Leakage inductance from the magnetic energy",
    "design_type": ["Maxwell 2D"],
    "fields_type": ["Fields"],
    "primary_sweep": "Distance",
    "assignment": "",
    "assignment_type": ["Line"],
    "operations": [
        "Fundamental_Quantity('Energy')",
        "EnterSurface('HV')",
        "Operation('SurfaceValue')",
        "Operation('Integrate')",
        "Scalar_Function(FuncValue='HV_mean_turn_length')",
        "Operation('*')",
        "Fundamental_Quantity('Energy')",
        "EnterSurface('LV')",
        "Operation('SurfaceValue')",
        "Operation('Integrate')",
        "Scalar_Function(FuncValue='LV_mean_turn_length')",
        "Operation('*')",
        "Fundamental_Quantity('Energy')",
        "EnterSurface('Region')",
        "Operation('SurfaceValue')",
        "Operation('Integrate')",
        "Scalar_Function(FuncValue='HV_LV_gap_length')",
        "Operation('*')",
        "Operation('+')",
        "Operation('+')",
        "Scalar_Constant(2)",
        "Operation('*')",
        "Scalar_Function(FuncValue='HV_current')",
        "Scalar_Function(FuncValue='HV_current')",
        "Operation('*')",
        "Operation('/')",
    ],
    "report": ["Data Table", "Rectangular Plot"],
}
m2d.post.fields_calculator.add_expression(leakage_inductance, None)

leakage_reactance = {
    "name": "Leakage_reactance",
    "description": "Leakage reactance from the magnetic energy",
    "design_type": ["Maxwell 2D"],
    "fields_type": ["Fields"],
    "primary_sweep": "Distance",
    "assignment": "",
    "assignment_type": ["Line"],
    "operations": [
        "NameOfExpression('Leakage_inductance')",
        "Scalar_Constant(2)",
        "Scalar_Constant(3.14159)",
        "Scalar_Function(FuncValue='Frequency')",
        "Operation('*')",
        "Operation('*')",
        "Operation('*')",
    ],
    "report": ["Data Table", "Rectangular Plot"],
}
m2d.post.fields_calculator.add_expression(leakage_reactance, None)

m2d.post.create_report(
    expressions=["Leakage_inductance", "Leakage_reactance"],
    report_category="Fields",
    primary_sweep_variable="core_width",
    plot_type="Data Table",
    plot_name="Transformer Leakage Inductance",
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmpclobvbu_.ansys\Magnetostatic.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmpclobvbu_.ansys\Magnetostatic.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.009378194808959961


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


PyAEDT WARNING: No context provided for Fields. Assigned object: None


Transformer Leakage Inductance

## Print leakage inductance and reactance values in AEDT Message Manager

In [12]:
m2d.logger.clear_messages()
m2d.logger.info(
    "Leakage_inductance =  {:.4f}H".format(
        m2d.post.get_scalar_field_value(quantity="Leakage_inductance")
    )
)
m2d.logger.info(
    "Leakage_reactance =  {:.2f}Ohm".format(
        m2d.post.get_scalar_field_value(quantity="Leakage_reactance")
    )
)

PyAEDT INFO: Exporting Leakage_inductance field. Be patient


PyAEDT INFO: Quantity Leakage_inductance not present. Trying to get it from Stack


PyAEDT INFO: Leakage_inductance =  0.1564H


PyAEDT INFO: Exporting Leakage_reactance field. Be patient


PyAEDT INFO: Quantity Leakage_reactance not present. Trying to get it from Stack


PyAEDT INFO: Leakage_reactance =  49.15Ohm


## Plot energy in the simulation domain

Most of the energy is confined in the air between the HV and LV windings.

In [13]:
energy_field_overlay = m2d.post.create_fieldplot_surface(
    assignment=m2d.modeler.object_names,
    quantity="energy",
    plot_name="Energy",
)

PyAEDT INFO: Active Design set to Transformer_leakage_inductance


## Release AEDT

In [14]:
m2d.save_project()
m2d.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project Magnetostatic Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [15]:
temp_folder.cleanup()